<a href="https://colab.research.google.com/github/kashfay110/stock-market-python/blob/main/svr_ann_collab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install mplfinance


     |████████████████████████████████| 61kB 3.9MB/s 


In [2]:
# import quandl
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
import pandas as pd
# from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import mplfinance as mpf
import keras
from keras.models import Sequential
from keras.layers import Dense

In [3]:
# Get the stock data
#df = quandl.get("WIKI/FB")
#df = pd.read_csv (r'dataset/FB.csv')
# Take a look at data
#print(df.head())

from google.colab import files
uploaded = files.upload()
import io
df = pd.read_csv(io.BytesIO(uploaded['FB.csv']))
df.head()

df2 = pd.read_csv(io.BytesIO(uploaded['FB.csv']))
#print(df2.head())
# Get the Adjusted Close
df = df[['Adj Close']]
# Take a look at new data
#print(df.head())

Saving FB.csv to FB.csv


In [4]:
# A variable for predicting 'forecast_out' days out in future
forecast_out = 1

In [5]:
# Create another column (dependent variable) shifted 'forecast_out' units up
df['Prediction'] = df[["Adj Close"]].shift(-forecast_out)
#print(df.tail())

In [6]:
# Create the independent dataset
# Convert the dataframe to numpy array
X = np.array(df.drop(['Prediction'],1))
# Remove the last 'forecast_out' rows
X = X[:-forecast_out]
#print(X)

In [7]:
# Create the dependent dataset
# Convert the dataframe to numpy array (All of the values including the NaNs)
y = np.array(df['Prediction'])
# Get all of the y values except the last 'forecast_out' rows
y = y[:-forecast_out]
#print(y)

In [8]:
z = np.array(df2)
z = z[:-forecast_out]
print(z)

[['2012-05-21' 36.529999 36.66 ... 34.029999 34.029999 168192700]
 ['2012-05-22' 32.610001000000004 33.59 ... 31.0 31.0 101786600]
 ['2012-05-23' 31.370001000000002 32.5 ... 32.0 32.0 73600000]
 ...
 ['2021-02-19' 269.859985 270.269989 ... 261.559998 261.559998 25549900]
 ['2021-02-22' 257.950012 263.070007 ... 260.329987 260.329987 17382300]
 ['2021-02-23' 259.5 267.459991 ... 265.859985 265.859985 21905400]]


In [9]:
# Split the data into 80% training and 20% testing
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
# train_set = x_train + y_train
# test_set = x_test + y_test

In [10]:
# Create and train the Support Vector Machine (Regressor)
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr_rbf.fit(x_train, y_train)

SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [11]:
# Create and train the Linear Regression model
lr = LinearRegression()
# Train model
lr.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [12]:
#Set x_forecast equal to the last forecast_out rows of the original dat set from Adj. Close column
x_forecast = np.array(df.drop(['Prediction'],1))[-forecast_out:]
#print(x_forecast)

In [13]:
# Print the SVR prediction for the next forecast_out days
svm_prediction = svr_rbf.predict(x_forecast)
print(f'The next {forecast_out} days by the SVR model: \n{svm_prediction}')

# Print the Linear Regression prediction for the next forecast_out days
lr_prediction = lr.predict(x_forecast)
print(f'The next {forecast_out} days by the LR model: \n{lr_prediction}')

The next 1 days by the SVR model: 
[261.89832359]
The next 1 days by the LR model: 
[264.26294377]


In [14]:
# Testing Model: Score returns the coefficient of determination R^2 of the prediction
# Best possible score is 1.0
svm_confidence = svr_rbf.score(x_test, y_test)
print("svr confidence: ", svm_confidence)

# Testing Model: Score returns the coefficient of determination R^2 of the prediction
# It provides a measure of how well observed outcomes are replicated by the model, based on the proportion of total variation of outcomes explained by the model
# Best possible score is 1.0
lr_confidence = lr.score(x_test, y_test)
print("lr confidence: ", lr_confidence)

svr confidence:  0.9978131850417562
lr confidence:  0.9983191212575176


In [15]:
## Creating predicted values for both models
y_pred_svr = svr_rbf.predict(x_test)
y_pred_lr = lr.predict(x_test)

In [16]:
model = Sequential()
model.add(Dense(200, input_dim=1, activation='relu'))
model.add(Dense(200, input_dim=200, activation='relu'))
model.add(Dense(200, input_dim=200, activation='relu'))
model.add(Dense(1, activation='linear'))

In [17]:
keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(loss='mean_squared_error', optimizer='RMSprop', metrics=['mean_absolute_percentage_error'])

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 200)               400       
_________________________________________________________________
dense_1 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_2 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 201       
Total params: 81,001
Trainable params: 81,001
Non-trainable params: 0
_________________________________________________________________


In [21]:
history = model.fit(x_train, y_train, epochs=100, batch_size=32,validation_split=0.15,verbose=1)

Epoch 1/100
47/47 [==============================] - 0s 4ms/step - loss: 14.7914 - mean_absolute_percentage_error: 2.1112 - val_loss: 15.4981 - val_mean_absolute_percentage_error: 2.1192
Epoch 2/100
47/47 [==============================] - 0s 3ms/step - loss: 15.1227 - mean_absolute_percentage_error: 2.0682 - val_loss: 22.4203 - val_mean_absolute_percentage_error: 3.0459
Epoch 3/100
47/47 [==============================] - 0s 3ms/step - loss: 15.7287 - mean_absolute_percentage_error: 2.1633 - val_loss: 11.3042 - val_mean_absolute_percentage_error: 1.9436
Epoch 4/100
47/47 [==============================] - 0s 3ms/step - loss: 15.5671 - mean_absolute_percentage_error: 2.0522 - val_loss: 30.8579 - val_mean_absolute_percentage_error: 3.1483
Epoch 5/100
47/47 [==============================] - 0s 3ms/step - loss: 16.3086 - mean_absolute_percentage_error: 2.1649 - val_loss: 19.9671 - val_mean_absolute_percentage_error: 2.4542
Epoch 6/100
47/47 [==============================] - 0s 3ms/step 

In [20]:
keras.backend.clear_session()